In [3]:
# NOTICE!!!!
# replace 'E' in the equation with 'e' as sympy would treat 'E' as numerical value e.
##################

import sympy as sp
import pandas as pd

# Define custom operations using subclassing of sympy.Function
class Add(sp.Function):
    @classmethod
    def eval(cls, x, y):
        return x + y

class Sub(sp.Function):
    @classmethod
    def eval(cls, x, y):
        return x - y

class Mul(sp.Function):
    @classmethod
    def eval(cls, x, y):
        return x * y

class Div(sp.Function):
    @classmethod
    def eval(cls, x, y):
        try:
            result = sp.Piecewise(
                (1, sp.Eq(y, 0)),  # Return 1 if y is exactly zero
                (1, sp.Eq(y, sp.zoo)),  # Return 1 if y is exactly zero
                (x / y, True)           # Normal division otherwise
            )
            return result.simplify()  # Simplify to handle potential further issues
        except Exception as e:
            print(f"Division error: {e}")
            return 1000  # Return NaN on error
        
# Define the custom operations using subclassing of sympy.Function
class Div_for_visualization(sp.Function):
    @classmethod
    def eval(cls, x, y):
        # Use Piecewise to handle division by zero robustly
        return x / y

class Neg(sp.Function):
    @classmethod
    def eval(cls, x):
        return -x
    

# Update converter for sympify to recognize custom classes
converter = {
    'add': Add,
    'sub': Sub,
    'mul': Mul,
    'div': Div,
    'neg': Neg,
}

# Update converter for sympify to recognize custom classes
converter_vis = {
    'add': Add,
    'sub': Sub,
    'mul': Mul,
    'div': Div_for_visualization,
    'neg': Neg,
}

# Function to apply the equation to each row of the DataFrame
def apply_equation(row):
    # Replace variables in the expression with row data
    local_env = {
        'Pr': row['Pr'],
        'La': row['La'],
        'Gd': row['Gd'],
        'Nd': row['Nd'],
        'Ce': row['Ce'],
        'Sm': row['Sm'],
        'Y' : row['Y'],
        "EN": row['EN'],
        'R' : row['R'],
        'IE': row['IE'],
        'LAS': row['LAS'],
        'M' : row['M'],
        'e' : row['E'],
    }
    try:
        result = expr.subs(local_env)
        # Force numeric evaluation if the result is not numeric
        if isinstance(result, sp.Expr):  # Check if result is still a symbolic expression
            result = result.evalf()  # Evaluate numerically
        # if result == sp.zoo:
        #     result = 1
    except Exception as e:
        print(f"Error evaluating row: {local_env}")
        print(f"Error message: {str(e)}")
        result = None  # Optional: handle error case by assigning a default value or None
    return result

    # print("Local Environment:", local_env)  # Debug: Print the local environment for a row
    # return expr.subs(local_env)

In [4]:
equ = "add(div(sub(mul(EN, div(Nd, Y)), Y), La), add(div(sub(Pr, e), 3.470), e))"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./all_rp650.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('all_rp650_pred.xlsx', index=False)

0.288184438040346*Pr + 0.711815561959654*e + (EN*Nd/Y - Y)/La


In [5]:
equ = "add(div(6.424, add(Sm, div(0.466, sub(La, R)))), EN)"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./La_rp650.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('La_rp650_pred.xlsx', index=False)

EN + 6.424/(Sm + 0.466/(La - R))


In [6]:
equ = "add(div(sub(Pr, Y), 3.488), add(div(div(sub(EN, IE), sub(EN, Sm)), Y), e))"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./Gd_rp650.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Gd_rp650_pred.xlsx', index=False)

0.286697247706422*Pr - 0.286697247706422*Y + e + (EN - IE)/(Y*(EN - Sm))


In [7]:
equ = "add(div(sub(6.163, Y), div(mul(sub(EN, Y), 7.642), sub(Y, La))), add(e, 0.212))"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./Nd_rp650.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Nd_rp650_pred.xlsx', index=False)

e + (6.163 - Y)*(-La + Y)/(7.642*EN - 7.642*Y) + 0.212


In [11]:
equ = "add(div(sub(e, M), 4.288), add(div(div(Gd, 4.886), mul(sub(LAS, M), add(La, La))), M))"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./Ce_rp650.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Ce_rp650_pred.xlsx', index=False)

0.102333196889071*Gd/(La*(LAS - M)) + 0.766791044776119*M + 0.233208955223881*e


In [9]:
equ = "add(div(3.325, mul(sub(R, Y), mul(Nd, IE))), add(div(sub(M, e), add(La, -2.843)), e))"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./Sm_rp650.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Sm_rp650_pred.xlsx', index=False)

e + (M - e)/(La - 2.843) + 3.325/(IE*Nd*(R - Y))


In [10]:
equ = "add(div(sub(e, add(sub(5.698, Ce), Y)), mul(sub(La, IE), Nd)), e)"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter)

# Read the Excel file
df = pd.read_excel('./Y_rp650.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Y_rp650_pred.xlsx', index=False)
# df['Pred']

e + (Ce - Y + e - 5.698)/(Nd*(-IE + La))
